In [31]:
import pandas as pd          
import numpy as np            
import math                 
import matplotlib.pyplot as plt 
import gurobipy as gp      
from gurobipy import GRB    

# Load datasets into dataframes
employment_data = pd.read_csv('/Users/nicholas/Desktop/Project Report 1 - Optimization/employment_rate.csv')
income_data = pd.read_csv('/Users/nicholas/Desktop/Project Report 1 - Optimization/avg_individual_income.csv')
population_data = pd.read_csv('/Users/nicholas/Desktop/Project Report 1 - Optimization/population.csv')
facility_data = pd.read_csv('/Users/nicholas/Desktop/Project Report 1 - Optimization/child_care_regulated.csv')

# Merge employment and income data on shared 'zipcode' column
combined_data = employment_data.merge(income_data, left_on='zipcode', right_on='ZIP code').drop(columns=['ZIP code'])

# Define demand levels based on employment rate and average income criteria
combined_data['demand_level'] = np.where(
    (combined_data['employment rate'] > 0.6) | (combined_data['average income'] < 60000),
    "High",
    "Normal"
)

# Display the columns of population_data to verify names after renaming attempt
print("Population Data Columns Before Renaming:", population_data.columns)

# Adjust column names in population data to match expected format if possible
population_data.rename(columns={'9-May': '5-9', '14-Oct': '10-14'}, inplace=True)

# Display columns again to confirm renaming
print("Population Data Columns After Renaming:", population_data.columns)

# Check if the expected columns are present before merging
if 'zipcode' in population_data.columns and 'Total' in population_data.columns:
    # Assume 'Total' is equivalent to '0-5' if necessary
    population_data_subset = population_data[['zipcode', 'Total', '-5', '5-9', '10-14']].copy()
    population_data_subset = population_data_subset.rename(columns={'Total': '0-5'})
else:
    print("Expected columns not found in population_data. Please verify column names.")
    population_data_subset = None  # Fallback in case of missing columns

# Merge with combined_data if the subset was successfully created
if population_data_subset is not None:
    combined_data = combined_data.merge(population_data_subset, on='zipcode', how='left')

    # Calculate estimated child population across age groups with a scaling factor
    child_scale = 13 / 15
    combined_data['estimated_children'] = np.round(
        child_scale * (combined_data['0-5'] + combined_data['5-9'] + combined_data['10-14'])
    )
else:
    print("Population data subset not created due to missing columns.")

# Summing capacity for ages 0-5 in each facility
facility_data['capacity_0_5'] = facility_data[['infant_capacity', 'toddler_capacity', 'preschool_capacity']].sum(axis=1)

# Aggregate facility capacity data by zip code for total and age-specific capacities
capacity_by_zip = facility_data.groupby('zip_code').agg(
    total_capacity=('total_capacity', 'sum'),
    total_0_5_capacity=('capacity_0_5', 'sum')
).reset_index()

# Integrate capacity data into the main dataframe
combined_data = combined_data.merge(capacity_by_zip, left_on='zipcode', right_on='zip_code', how='left')

# Define if each area qualifies as a "child care desert" based on demand level and capacity thresholds
combined_data['childcare_desert'] = np.where(
    combined_data['demand_level'] == 'High',
    np.where(combined_data['total_capacity'] <= 0.5 * combined_data['estimated_children'], "Yes", "No"),
    np.where(combined_data['total_capacity'] <= (1/3) * combined_data['estimated_children'], "Yes", "No")
)

# Calculate the capacity shortfall for areas identified as child care deserts
combined_data['capacity_gap'] = np.where(
    combined_data['childcare_desert'] == "Yes",
    np.where(
        combined_data['demand_level'] == 'High',
        np.ceil(0.5 * combined_data['estimated_children']) - combined_data['total_capacity'],
        np.ceil((1/3) * combined_data['estimated_children']) - combined_data['total_capacity']
    ),
    0
)

# Export the processed data to a new CSV file for further analysis
combined_data.to_csv('/Users/nicholas/Desktop/Project Report 1 - Optimization/processed_data.csv', index=False)

# Create a dictionary for facilities grouped by zip code to organize budget calculations
facilities_by_zip = {
    zip_code: facility_data[facility_data['zip_code'] == zip_code].to_dict(orient='records')
    for zip_code in combined_data['zipcode'].unique()
}

# Display the first few rows of the final dataframe
combined_data.head()

Population Data Columns Before Renaming: Index(['zipcode', 'Total', '-5', '5-9', '10-14', '15-19', '20-24', '25-29',
       '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69',
       '70-74', '75-79', '80-84', '85+'],
      dtype='object')
Population Data Columns After Renaming: Index(['zipcode', 'Total', '-5', '5-9', '10-14', '15-19', '20-24', '25-29',
       '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69',
       '70-74', '75-79', '80-84', '85+'],
      dtype='object')


,zipcode,employment rate,average income,demand_level,0-5,-5,5-9,10-14,estimated_children,zip_code,total_capacity,total_0_5_capacity,childcare_desert,capacity_gap
0,10001,0.595097,102878.033603,Normal,27004,744,784,942,24899.0,10001.0,609.0,0.0,Yes,7691.0
1,10002,0.520662,59604.041165,High,76518,2142,3046,3198,71727.0,10002.0,4729.0,18.0,Yes,31135.0
2,10003,0.497244,114273.049645,Normal,53877,1440,1034,953,48415.0,10003.0,1995.0,0.0,Yes,14144.0
3,10004,0.506661,132004.310345,Normal,4579,433,182,161,4266.0,10004.0,263.0,0.0,Yes,1159.0
4,10005,0.665833,121437.713311,High,8801,484,204,229,8003.0,10005.0,39.0,0.0,Yes,3963.0


In [33]:
# Import Gurobi library
import gurobipy as gp
from gurobipy import GRB

# Initialize Gurobi model
m = gp.Model("budgeting_model")

# Create decision variables
# y variables for expanding existing facilities: one variable for each age group per facility in a given zipcode
y = {}
for zipcode in budget_dict.keys():
    for facility in budget_dict[zipcode]:
        if facility['total_capacity'] <= 500:  # Only facilities with <= 500 slots can be expanded
            for age_group in ['5p', '5m']:
                y[(zipcode, facility['facility_id'], age_group)] = m.addVar(
                    vtype=GRB.INTEGER, name=f"y_{zipcode}_{facility['facility_id']}_{age_group}"
                )

# x variables for building new facilities: one variable for each age group per facility type in each zipcode
x = {}
for zipcode in budget_dict.keys():
    for facility_size in ['small', 'medium', 'large']:
        for age_group in ['5p', '5m']:
            x[(zipcode, facility_size, age_group)] = m.addVar(
                vtype=GRB.INTEGER, name=f"x_{zipcode}_{facility_size}_{age_group}"
            )

# Fill NaN values in capacity columns with zero to avoid calculation errors
combined_data['total_capacity'] = combined_data['total_capacity'].fillna(0)

# Use total_0_5_capacity instead of total_capacity_(0-5)
if 'total_0_5_capacity' in combined_data.columns:
    combined_data['total_0_5_capacity'] = combined_data['total_0_5_capacity'].fillna(0)
else:
    print("'total_0_5_capacity' column not found. Please check data merging step.")

# Add constraints
# Constraint 1: Ensure newly created slots (expansions + new facilities) meet the gap for each zipcode
for zipcode in budget_dict.keys(): 
    m.addConstr(
        gp.quicksum(
            y[zipcode, facility['facility_id'], age] 
            for facility in budget_dict[zipcode] if facility['total_capacity'] <= 500 for age in ['5p', '5m']
        ) + 
        gp.quicksum(
            x[zipcode, facility, age] 
            for facility in ['small', 'medium', 'large'] for age in ['5p', '5m']
        ) >= combined_data.loc[combined_data['zipcode'] == zipcode, 'capacity_gap'].values[0],
        f"fill_gap_{zipcode}"
    )

# Big M constant for constraints
bigM = 1.0

# Constraint 2: Expansion limit <= 20% of initial slots for each facility
for zipcode in budget_dict.keys(): 
    for facility in budget_dict[zipcode]:
        if facility['total_capacity'] > 0 and facility['total_capacity'] < 500:
            current_capacity = facility['total_capacity']
            total_expanded_slots = gp.quicksum(y[zipcode, facility['facility_id'], age] for age in ['5p', '5m'])
            m.addConstr(total_expanded_slots <= 0.2 * current_capacity, f"expansion_limit_{zipcode}_{facility['facility_id']}")

# Constraint 3: Total expanded slots + current capacity <= 500 slots per facility
for zipcode in budget_dict.keys(): 
    for facility in budget_dict[zipcode]:
        if facility['total_capacity'] > 0 and facility['total_capacity'] < 500:
            current_capacity = facility['total_capacity']
            total_expanded_slots = gp.quicksum(y[zipcode, facility['facility_id'], age] for age in ['5p', '5m'])
            m.addConstr(total_expanded_slots + current_capacity <= 500, f"max_500_slots_{zipcode}_{facility['facility_id']}")

# Define the objective function z
z = 0

# Cost calculation for new facilities (x variables)
for zipcode in budget_dict.keys(): 
    z += gp.quicksum(
        x[zipcode, 'small', age] * (65000 / 100) for age in ['5p', '5m']
    ) + gp.quicksum(
        x[zipcode, 'medium', age] * (115000 / 400) for age in ['5p', '5m']
    ) + gp.quicksum(
        x[zipcode, 'large', age] * (95000 / 200) for age in ['5p', '5m']
    ) + gp.quicksum(
        x[zipcode, facility, '5m'] * 100 for facility in ['small', 'medium', 'large']
    )

# Cost calculation for expansions (y variables)
for zipcode in budget_dict.keys(): 
    for facility in budget_dict[zipcode]:
        if facility['total_capacity'] > 0 and facility['total_capacity'] <= 500:
            expansion_cost = 20000 + facility['total_capacity'] * 200
            expansion_ratio = gp.quicksum(y[zipcode, facility['facility_id'], age] for age in ['5p', '5m']) / facility['total_capacity']
            z += expansion_cost * expansion_ratio + gp.quicksum(y[zipcode, facility['facility_id'], '5m']) * 100

# Set objective to minimize total cost
m.setObjective(z, GRB.MINIMIZE)

# Optimize the model
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 0 columns and 0 nonzeros
Model fingerprint: 0xf9715da1
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [35]:
# Initialize a dummy variable to accumulate results from the optimized decision variables
dummy = 0

# Retrieve and print values of decision variables from the Gurobi model after optimization
for v in m.getVars():
    print(f'{v.varName}: {v.x}')
    dummy += v.x

# Calculate and print the total gap from the combined_data dataframe
total_gap = combined_data['capacity_gap'].sum()
print("Total Capacity Gap:", total_gap)
print("Sum of Decision Variable Values:", dummy)

# Loop over each zip code in the budget dictionary to update the dataframe with new slots and gaps
for zipcode_item in budget_dict.keys():
    dummy = 0
    
    # Sum the slots created by new facilities for each age group in the zip code
    dummy += gp.quicksum(
        x[zipcode_item, facility_size, age].x 
        for facility_size in ['small', 'medium', 'large'] 
        for age in ['5p', '5m']
    )
    
    # Add the slots expanded in existing facilities for each age group
    for facility in budget_dict[zipcode_item]:
        if facility['total_capacity'] > 0 and facility['total_capacity'] < 500:
            dummy += gp.quicksum(
                y[zipcode_item, facility['facility_id'], age].x 
                for age in ['5p', '5m']
            )
    
    # Check if the initial capacity gap is greater than zero and update 'new_slots' and 'new_gap'
    if combined_data.loc[combined_data['zipcode'] == zipcode_item, 'capacity_gap'].values[0] > 0:
        combined_data.loc[combined_data['zipcode'] == zipcode_item, 'new_slots'] = dummy
        combined_data.loc[combined_data['zipcode'] == zipcode_item, 'new_gap'] = (
            combined_data.loc[combined_data['zipcode'] == zipcode_item, 'capacity_gap'] - dummy
        )
    else:
        combined_data.loc[combined_data['zipcode'] == zipcode_item, 'new_gap'] = 0

# Display the updated dataframe with new slots and gaps
combined_data.head()


Total Capacity Gap: 6547326.0
Sum of Decision Variable Values: 0


,zipcode,employment rate,average income,demand_level,0-5,-5,5-9,10-14,estimated_children,zip_code,total_capacity,total_0_5_capacity,childcare_desert,capacity_gap
0,10001,0.595097,102878.033603,Normal,27004,744,784,942,24899.0,10001.0,609.0,0.0,Yes,7691.0
1,10002,0.520662,59604.041165,High,76518,2142,3046,3198,71727.0,10002.0,4729.0,18.0,Yes,31135.0
2,10003,0.497244,114273.049645,Normal,53877,1440,1034,953,48415.0,10003.0,1995.0,0.0,Yes,14144.0
3,10004,0.506661,132004.310345,Normal,4579,433,182,161,4266.0,10004.0,263.0,0.0,Yes,1159.0
4,10005,0.665833,121437.713311,High,8801,484,204,229,8003.0,10005.0,39.0,0.0,Yes,3963.0


In [39]:
# Initialize 'new_gap' with zeros if it doesn't already exist in the dataframe
if 'new_gap' not in combined_data.columns:
    combined_data['new_gap'] = 0

# Ensure that 'new_gap' contains only numeric data, filtering out any non-numeric types
combined_data['new_gap'] = pd.to_numeric(combined_data['new_gap'], errors='coerce')
combined_data['new_gap'] = combined_data['new_gap'].fillna(0)

# Safely get the maximum value of 'new_gap'
print("Maximum New Gap:", combined_data['new_gap'].max())

# Re-optimize the model if needed
m.optimize()

# Add constraints to ensure total slots for ages 0-5 meet two-thirds of the population requirement for each zipcode
for zipcode_item in budget_dict.keys(): 
    m.addConstr(
        gp.quicksum(
            y[zipcode_item, facility['facility_id'], age] 
            for facility in budget_dict[zipcode_item] 
            if facility['total_capacity'] > 0 and facility['total_capacity'] < 500 
            for age in ['5m']
        ) +
        gp.quicksum(
            x[zipcode_item, facility_size, age] 
            for facility_size in ['small', 'medium', 'large'] 
            for age in ['5m']
        ) +
        combined_data.loc[combined_data['zipcode'] == zipcode_item, 'total_0_5_capacity'].values[0] >=
        (2/3) * combined_data.loc[combined_data['zipcode'] == zipcode_item, '0-5'].values[0],
        f"min_capacity_{zipcode_item}"
    )


Maximum New Gap: 0
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 0 columns and 0 nonzeros
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [41]:
# Initialize a dummy variable to accumulate results from the optimized decision variables
dummy = 0

# Retrieve and print values of decision variables from the Gurobi model after optimization
for v in m.getVars():
    print(f'{v.varName}: {v.x}')
    dummy += v.x

# Reinitialize dummy variable for second iteration of printing decision variables
dummy = 0
for v in m.getVars():
    print(f'{v.varName}: {v.x}')
    dummy += v.x

# Loop over each zip code in the budget dictionary to update the dataframe with new slots and gaps
for zipcode_item in budget_dict.keys():
    dummy = 0
    
    # Sum the slots created by new facilities for each age group in the zip code
    dummy += gp.quicksum(
        x[zipcode_item, facility_size, age].x 
        for facility_size in ['small', 'medium', 'large'] 
        for age in ['5p', '5m']
    )
    
    # Add the slots expanded in existing facilities for each age group
    for facility in budget_dict[zipcode_item]:
        if facility['total_capacity'] > 0 and facility['total_capacity'] < 500:
            dummy += gp.quicksum(
                y[zipcode_item, facility['facility_id'], age].x 
                for age in ['5p', '5m']
            )
    
    # Update 'new_gap' and 'new_slots' based on the capacity gap
    if combined_data.loc[combined_data['zipcode'] == zipcode_item, 'capacity_gap'].values[0] > 0:
        combined_data.loc[combined_data['zipcode'] == zipcode_item, 'new_gap'] = (
            combined_data.loc[combined_data['zipcode'] == zipcode_item, 'capacity_gap'] - dummy
        )
        combined_data.loc[combined_data['zipcode'] == zipcode_item, 'new_slots'] = dummy
    else:
        combined_data.loc[combined_data['zipcode'] == zipcode_item, 'new_gap'] = 0

# Display the updated dataframe with new slots and gaps
combined_data.head()


,zipcode,employment rate,average income,demand_level,0-5,-5,5-9,10-14,estimated_children,zip_code,total_capacity,total_0_5_capacity,childcare_desert,capacity_gap,new_gap
0,10001,0.595097,102878.033603,Normal,27004,744,784,942,24899.0,10001.0,609.0,0.0,Yes,7691.0,0
1,10002,0.520662,59604.041165,High,76518,2142,3046,3198,71727.0,10002.0,4729.0,18.0,Yes,31135.0,0
2,10003,0.497244,114273.049645,Normal,53877,1440,1034,953,48415.0,10003.0,1995.0,0.0,Yes,14144.0,0
3,10004,0.506661,132004.310345,Normal,4579,433,182,161,4266.0,10004.0,263.0,0.0,Yes,1159.0,0
4,10005,0.665833,121437.713311,High,8801,484,204,229,8003.0,10005.0,39.0,0.0,Yes,3963.0,0


In [43]:
# Initialize dictionaries for binary variables that categorize expansion ratios
is_less_than_10 = {}
is_between_10_and_15 = {}
is_greater_than_15 = {}

# Define binary variables for expansion categories
for zipcode_item in budget_dict.keys():
    for facility in budget_dict[zipcode_item]:
        if facility['total_capacity'] <= 500:
            # Create binary variables for the conditions
            is_less_than_10[(zipcode_item, facility['facility_id'])] = m.addVar(vtype=GRB.BINARY, name=f"is_less_than_10_{zipcode_item}_{facility['facility_id']}")
            is_between_10_and_15[(zipcode_item, facility['facility_id'])] = m.addVar(vtype=GRB.BINARY, name=f"is_between_10_and_15_{zipcode_item}_{facility['facility_id']}")
            is_greater_than_15[(zipcode_item, facility['facility_id'])] = m.addVar(vtype=GRB.BINARY, name=f"is_greater_than_15_{zipcode_item}_{facility['facility_id']}")

# Define a large constant for Big-M method constraints
bigM = 1.0

# Add constraints for each facility to limit the expansion based on current capacity and conditions
for zipcode_item in budget_dict.keys():
    for facility in budget_dict[zipcode_item]:
        if facility['total_capacity'] > 0 and facility['total_capacity'] < 500:
            # Get the current capacity of the facility from combined_data
            current_capacity = combined_data.loc[combined_data['facility_id'] == facility['facility_id'], 'total_capacity'].values
            total_y = gp.quicksum(y[zipcode_item, facility['facility_id'], age] for age in ['5p', '5m'])
            
            # Check if the current capacity exists
            if len(current_capacity) > 0:
                current_capacity_value = current_capacity[0]
                ratio = total_y / current_capacity_value

                # Ensure only one of the binary variables is active
                m.addConstr(is_less_than_10[(zipcode_item, facility['facility_id'])] + \
                            is_between_10_and_15[(zipcode_item, facility['facility_id'])] + \
                            is_greater_than_15[(zipcode_item, facility['facility_id'])] == 1)
                
                # Add constraints to enforce expansion cost tiers based on ratio
                m.addConstr(ratio <= 0.1 + (1 - is_less_than_10[(zipcode_item, facility['facility_id'])]) * bigM)
                m.addConstr(ratio >= 0.1 * is_between_10_and_15[(zipcode_item, facility['facility_id'])])
                m.addConstr(ratio <= 0.15 + (1 - is_between_10_and_15[(zipcode_item, facility['facility_id'])]) * bigM)
                m.addConstr(ratio >= 0.15 * is_greater_than_15[(zipcode_item, facility['facility_id'])])

# Initialize the objective function z for total cost calculation
z = 0

# Calculate cost for constructing new facilities based on facility size and age group
for zipcode_item in budget_dict.keys(): 
    z += gp.quicksum(x[zipcode_item, facility, age] * (65000 / 100) for facility in ['small'] for age in ['5p', '5m']) + \
         gp.quicksum(x[zipcode_item, facility, age] * (115000 / 400) for facility in ['medium'] for age in ['5p', '5m']) + \
         gp.quicksum(x[zipcode_item, facility, age] * (95000 / 200) for facility in ['large'] for age in ['5p', '5m']) + \
         gp.quicksum(x[zipcode_item, facility, '5m'] * 100 for facility in ['small', 'medium', 'large'])

# Calculate expansion costs based on the identified expansion categories
for zipcode_item in budget_dict.keys(): 
    for facility in budget_dict[zipcode_item]:
        if facility['total_capacity'] > 0 and facility['total_capacity'] <= 500:
            total_y = gp.quicksum(y[zipcode_item, facility['facility_id'], age] for age in ['5p', '5m'])
            ratio = total_y / facility['total_capacity']
            
            # Sum the costs based on the active binary variable condition for each expansion tier
            z += (
                is_less_than_10[(zipcode_item, facility['facility_id'])] * 
                ((20000 + facility['total_capacity'] * 200) * ratio +
                 gp.quicksum(y[zipcode_item, facility['facility_id'], '5m']) * 100) +
                is_between_10_and_15[(zipcode_item, facility['facility_id'])] * 
                ((20000 + facility['total_capacity'] * 400) * ratio +
                 gp.quicksum(y[zipcode_item, facility['facility_id'], '5m']) * 100) +
                is_greater_than_15[(zipcode_item, facility['facility_id'])] * 
                ((20000 + facility['total_capacity'] * 1000) * ratio +
                 gp.quicksum(y[zipcode_item, facility['facility_id'], '5m']) * 100)
            )

# Set the objective to minimize total cost
m.setObjective(z, GRB.MINIMIZE)

# Optimize the model
m.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 0 columns and 0 nonzeros
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  0.000000000e+00


In [45]:
# Dictionary to store slot-to-child ratios for each zip code
ratio = {}

# Calculate slot-to-child ratios for each zip code and create a Gurobi variable for it
for zipcode_item in budget_dict.keys():
    # Calculate the total slots after expansion and new facilities for each age group
    total_slots = gp.quicksum(
        y[zipcode_item, facility['facility_id'], age] for facility in budget_dict[zipcode_item] if facility['total_capacity'] <= 500 for age in ['5p', '5m']
    ) + gp.quicksum(
        x[zipcode_item, facility_size, age] for facility_size in ['small', 'medium', 'large'] for age in ['5p', '5m']
    ) + combined_data.loc[combined_data['zipcode'] == zipcode_item, 'total_capacity'].values[0]
    
    # Calculate the child population in each zip code
    child_population = combined_data.loc[combined_data['zipcode'] == zipcode_item, 'estimated_children'].values[0]
    
    # Define a Gurobi variable for the ratio of slots to child population
    ratio[zipcode_item] = m.addVar(vtype=GRB.CONTINUOUS, name=f"ratio_{zipcode_item}")
    
    # Set the ratio value as a constraint
    if child_population > 0:  # To avoid division by zero
        m.addConstr(ratio[zipcode_item] == total_slots / child_population, name=f"ratio_constraint_{zipcode_item}")
    else:
        m.addConstr(ratio[zipcode_item] == 0, name=f"ratio_constraint_{zipcode_item}")  # Set ratio to 0 if no children are present


In [47]:
# Auxiliary variable to represent the difference in ratios between any two zip codes
ratio_diff = {}

# Create fairness constraints between every pair of zip codes
zip_codes = list(budget_dict.keys())
for i in range(len(zip_codes)):
    for j in range(i + 1, len(zip_codes)):  # Ensures each pair is considered only once
        zipcode_1 = zip_codes[i]
        zipcode_2 = zip_codes[j]

        # Define an auxiliary variable to represent the absolute difference in ratios
        ratio_diff[(zipcode_1, zipcode_2)] = m.addVar(vtype=GRB.CONTINUOUS, name=f"ratio_diff_{zipcode_1}_{zipcode_2}")
        
        # Add constraints to set ratio_diff as the absolute difference between ratios of two zip codes
        m.addConstr(ratio_diff[(zipcode_1, zipcode_2)] >= ratio[zipcode_1] - ratio[zipcode_2], name=f"fairness_pos_diff_{zipcode_1}_{zipcode_2}")
        m.addConstr(ratio_diff[(zipcode_1, zipcode_2)] >= ratio[zipcode_2] - ratio[zipcode_1], name=f"fairness_neg_diff_{zipcode_1}_{zipcode_2}")
        
        # Constraint to ensure that the absolute difference does not exceed 0.1
        m.addConstr(ratio_diff[(zipcode_1, zipcode_2)] <= 0.1, name=f"fairness_constraint_{zipcode_1}_{zipcode_2}")


In [51]:
# Dictionary to store the social coverage for each zip code
coverage_0_5 = {}
coverage_total = {}

# Define the social coverage index variables for each zip code
for zipcode_item in budget_dict.keys():
    # Calculate the total slots available for age 0-5 in each zip code
    total_0_5_slots = gp.quicksum(
        y[zipcode_item, facility['facility_id'], '5p'] for facility in budget_dict[zipcode_item] if facility['total_capacity'] <= 500
    ) + gp.quicksum(
        x[zipcode_item, facility_size, '5p'] for facility_size in ['small', 'medium', 'large']
    ) + combined_data.loc[combined_data['zipcode'] == zipcode_item, 'total_0_5_capacity'].values[0]
    
    # Calculate the total slots for all children (0-12 age group)
    total_all_slots = gp.quicksum(
        y[zipcode_item, facility['facility_id'], age] for facility in budget_dict[zipcode_item] if facility['total_capacity'] <= 500 for age in ['5p', '5m']
    ) + gp.quicksum(
        x[zipcode_item, facility_size, age] for facility_size in ['small', 'medium', 'large'] for age in ['5p', '5m']
    ) + combined_data.loc[combined_data['zipcode'] == zipcode_item, 'total_capacity'].values[0]
    
    # Population data for age groups 0-5 and 0-12 in each zip code
    population_0_5 = combined_data.loc[combined_data['zipcode'] == zipcode_item, '0-5'].values[0]
    population_all = combined_data.loc[combined_data['zipcode'] == zipcode_item, 'estimated_children'].values[0]
    
    # Define Gurobi variables for coverage ratios (0-5 and total)
    coverage_0_5[zipcode_item] = m.addVar(vtype=GRB.CONTINUOUS, name=f"coverage_0_5_{zipcode_item}")
    coverage_total[zipcode_item] = m.addVar(vtype=GRB.CONTINUOUS, name=f"coverage_total_{zipcode_item}")
    
    # Set coverage ratios as the proportion of slots to population, ensuring no division by zero
    if population_0_5 > 0:
        m.addConstr(coverage_0_5[zipcode_item] == total_0_5_slots / population_0_5, name=f"coverage_0_5_constraint_{zipcode_item}")
    else:
        m.addConstr(coverage_0_5[zipcode_item] == 0, name=f"coverage_0_5_constraint_{zipcode_item}")
        
    if population_all > 0:
        m.addConstr(coverage_total[zipcode_item] == total_all_slots / population_all, name=f"coverage_total_constraint_{zipcode_item}")
    else:
        m.addConstr(coverage_total[zipcode_item] == 0, name=f"coverage_total_constraint_{zipcode_item}")

# Define the overall social coverage index with a 2:1 weighting for children aged 0-5
social_coverage_index = gp.quicksum(2 * coverage_0_5[zipcode_item] + coverage_total[zipcode_item] for zipcode_item in budget_dict.keys())

# Update the objective function to maximize the social coverage index while minimizing costs
m.setObjective(social_coverage_index, GRB.MAXIMIZE)


In [53]:
# Define the budget limit (in dollars)
budget_limit = 100_000_000  # $100 million

# Add the budget constraint to ensure the total cost does not exceed the budget
m.addConstr(z <= budget_limit, name="budget_constraint")

# Optimize the model with the budget constraint in place
m.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 0 columns and 0 nonzeros
Model fingerprint: 0x0268838a
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolve: All rows and columns removed
       0   -0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective -0.000000000e+00


In [57]:
# Display the optimized social coverage index value
print("Optimized Social Coverage Index:", m.objVal)

# Display total cost by directly evaluating the value of z (since z is not a Gurobi variable here)
total_cost = sum(
    x[zipcode_item, facility, age].x * (65000 / 100 if facility == 'small' else
                                        115000 / 400 if facility == 'medium' else
                                        95000 / 200)
    for zipcode_item in budget_dict.keys()
    for facility in ['small', 'medium', 'large']
    for age in ['5p', '5m']
) + sum(
    y[zipcode_item, facility['facility_id'], age].x * (200 if ratio <= 0.1 else
                                                       400 if 0.1 < ratio <= 0.15 else
                                                       1000)
    for zipcode_item in budget_dict.keys()
    for facility in budget_dict[zipcode_item]
    if facility['total_capacity'] > 0 and facility['total_capacity'] <= 500
    for age in ['5p', '5m']
)

print("Total Cost:", total_cost)

# Display child care slots allocated per zip code to observe the distribution
for zipcode_item in budget_dict.keys():
    total_slots_0_5 = coverage_0_5[zipcode_item].x * combined_data.loc[combined_data['zipcode'] == zipcode_item, '0-5'].values[0]
    total_slots_all = coverage_total[zipcode_item].x * combined_data.loc[combined_data['zipcode'] == zipcode_item, 'estimated_children'].values[0]
    
    print(f"Zip Code: {zipcode_item}")
    print(f"  Total Slots for Age 0-5: {total_slots_0_5}")
    print(f"  Total Slots for All Children: {total_slots_all}")
    print(f"  Slot-to-Child Ratio for Age 0-5: {coverage_0_5[zipcode_item].x}")
    print(f"  Slot-to-Child Ratio for All Children: {coverage_total[zipcode_item].x}")


Optimized Social Coverage Index: -0.0
Total Cost: 0


In [59]:
# Check fairness compliance by calculating the difference in slot-to-child ratios between each zip code pair
fairness_violations = []
zip_codes = list(budget_dict.keys())

for i in range(len(zip_codes)):
    for j in range(i + 1, len(zip_codes)):  # Only unique pairs
        zipcode_1 = zip_codes[i]
        zipcode_2 = zip_codes[j]
        
        # Calculate the absolute difference in ratios
        ratio_diff = abs(ratio[zipcode_1].x - ratio[zipcode_2].x)
        
        # Check if the difference exceeds 0.1
        if ratio_diff > 0.1:
            fairness_violations.append((zipcode_1, zipcode_2, ratio_diff))

# Report any fairness violations found
if fairness_violations:
    print("Fairness constraint violations found between zip codes (difference > 0.1):")
    for violation in fairness_violations:
        print(f"  Zip codes: {violation[0]} & {violation[1]}, Difference: {violation[2]}")
else:
    print("Fairness constraints satisfied: No violations found.")


Fairness constraints satisfied: No violations found.


In [61]:
# Check if the total cost is within the budget
if total_cost <= budget_limit:
    print(f"Budget compliance satisfied: Total cost ({total_cost}) is within the $100 million budget.")
else:
    print(f"Budget compliance violation: Total cost ({total_cost}) exceeds the $100 million budget.")


Budget compliance satisfied: Total cost (0) is within the $100 million budget.


In [63]:
# Analyze social coverage distribution by printing slots and ratios for each zip code
print("\nSocial Coverage Distribution by Zip Code:")
for zipcode_item in budget_dict.keys():
    total_slots_0_5 = coverage_0_5[zipcode_item].x * combined_data.loc[combined_data['zipcode'] == zipcode_item, '0-5'].values[0]
    total_slots_all = coverage_total[zipcode_item].x * combined_data.loc[combined_data['zipcode'] == zipcode_item, 'estimated_children'].values[0]
    
    print(f"\nZip Code: {zipcode_item}")
    print(f"  Total Slots for Age 0-5: {total_slots_0_5}")
    print(f"  Total Slots for All Children: {total_slots_all}")
    print(f"  Slot-to-Child Ratio for Age 0-5: {coverage_0_5[zipcode_item].x}")
    print(f"  Slot-to-Child Ratio for All Children: {coverage_total[zipcode_item].x}")



Social Coverage Distribution by Zip Code:
